In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE201525"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE201525"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE201525.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE201525.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE201525.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE201525-GPL21810_series_matrix.txt.gz', 'GSE201525_family.soft.gz']
SOFT file: ../../input/GEO/Ovarian_Cancer/GSE201525/GSE201525_family.soft.gz
Matrix file: ../../input/GEO/Ovarian_Cancer/GSE201525/GSE201525-GPL21810_series_matrix.txt.gz
Background Information:
!Series_title	"Investigation of the anti-tumour properties of interferon epsilon in high grade serous ovarian cancer"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['treatment: UT', 'treatment: 1000IU_IFNE', 'treatment: 100IU_IFNE', 'treatment: 10IU_IFNE', 'treatment: 1000IU_IFNB', 'treatment: 100IU_IFNB', 'treatment: 10IU_IFNB'], 1: ['replicate: R1', 'replicate: R2', 'replicate: R3']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Assess gene expression data availability
# Based on the provided information, we cannot determine if gene expression data is present
# The Series title suggests this might be interferon treatment data, but we don't have explicit confirmation
# Let's set this to False until we can verify gene expression data exists
is_gene_available = False

# 2. Variable availability and data type conversion
# From the sample characteristics dictionary, we can see:
# - There are treatment groups (UT, IFNE at different doses, IFNB at different doses)
# - There are replicates (R1, R2, R3)
# But no direct information about Ovarian Cancer status, age, or gender

# 2.1 Data Availability
# None of the required variables appear to be directly available in the sample characteristics
trait_row = None  # No direct ovarian cancer status information
age_row = None    # No age information
gender_row = None # No gender information

# 2.2 Data Type Conversion Functions
# Although we don't have these data, we'll define conversion functions as required

def convert_trait(value):
    """Convert trait value to binary format (0 or 1)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    value_lower = str(value).lower()
    if 'control' in value_lower or 'normal' in value_lower or 'healthy' in value_lower:
        return 0
    elif 'cancer' in value_lower or 'tumor' in value_lower or 'oc' in value_lower:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to numeric format"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        age = float(value)
        return age
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary format (0=female, 1=male)"""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    value_lower = str(value).lower()
    if 'f' in value_lower or 'female' in value_lower:
        return 0
    elif 'm' in value_lower or 'male' in value_lower:
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is not available (trait_row is None)
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
Found potential subseries references:
!Series_relation = SuperSeries of: GSE201337
!Series_relation = SuperSeries of: GSE201345
!Series_relation = SuperSeries of: GSE215261



Gene data extraction result:
Number of rows: 62976
First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the observed identifiers, these appear to be just numeric values
# (1, 2, 3, 4, etc.) rather than standard human gene symbols
# These are likely probe IDs or feature indices that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_51_P399985', 'A_55_P2508138'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', nan, nan], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015742', 'NR_028378'], 'GB_ACC': [nan, nan, nan, 'NM_015742', 'NR_028378'], 'LOCUSLINK_ID': [nan, nan, nan, 17925.0, 100034739.0], 'GENE_SYMBOL': [nan, nan, nan, 'Myo9b', 'Gm17762'], 'GENE_NAME': [nan, nan, nan, 'myosin IXb', 'predicted gene, 17762'], 'UNIGENE_ID': [nan, nan, nan, 'Mm.33779', 'Mm.401643'], 'ENSEMBL_ID': [nan, nan, nan, 'ENSMUST00000170242', nan], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015742|ref|NM_001142322|ref|NM_001142323|ens|ENSMUST00000170242', 'ref|NR_028378|gb|AK171729|gb|AK045818|gb|AK033161'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'chr8:73884459-73884518', 'chr2:17952143-17952202'

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# The 'ID' column in gene_annotation contains numeric identifiers matching expression data
# The 'GENE_SYMBOL' column contains the gene symbols we want to map to
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# 2. Get gene mapping dataframe with the two identified columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
# Here we handle many-to-many relations between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the results
print(f"Original gene expression data shape: {gene_data.shape}")
print(f"Number of unique gene symbols after mapping: {len(gene_data.index.unique())}")
print("First 10 gene symbols:")
print(gene_data.index[:10])


Original gene expression data shape: (511, 24)
Number of unique gene symbols after mapping: 511
First 10 gene symbols:
Index(['A130033P14', 'A230055C15', 'A330044H09', 'A430057O09', 'A430085C19',
       'A530028O18', 'A830011I04', 'AA060545', 'AA066038', 'AA386476'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols
import numpy as np

# Get mapping using the correct column names as identified in Step 6
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# First normalize gene symbols in the gene expression data
try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {gene_data_normalized.shape}")
except Exception as e:
    print(f"Error during normalization: {e}")
    # If normalization fails, continue with the original mapped data
    gene_data_normalized = gene_data

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Given the information we've gathered, this is a SuperSeries dataset that doesn't 
# have appropriate clinical data for Ovarian Cancer studies
# Create a minimal clinical dataframe with missing trait values
sample_ids = gene_data.columns.tolist()
print(f"Sample IDs from gene data: {len(sample_ids)} samples")

# Create an empty clinical dataframe
# Since we don't have proper trait information, we'll set all values to NaN
clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
clinical_df.loc[trait] = np.nan  # Using NaN to indicate missing trait information

print(f"Clinical data shape: {clinical_df.shape}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
# Skip missing value handling if all trait values are NaN
if clinical_df.loc[trait].isna().all():
    print("No valid trait values available, skipping missing value handling")
    linked_data_cleaned = linked_data
else:
    print("Handling missing values...")
    linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Since all samples have missing trait values, the trait is considered biased
# In this case, we don't need to call judge_and_remove_biased_features
is_trait_biased = True

# 6. Validate the dataset and save cohort information
note = "This is a SuperSeries (GSE201525) containing multiple subseries. No direct trait information for Ovarian Cancer is available in this dataset. The gene expression data can be extracted but lacks appropriate clinical annotation for case-control analysis."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # No trait information available
    is_biased=is_trait_biased,
    df=linked_data_cleaned,
    note=note
)

# 7. Save the linked data if it's usable (which it won't be due to missing trait info)
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_cleaned.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to missing trait information. Final linked data not saved.")

After normalization: (30, 24)
Gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE201525.csv
Sample IDs from gene data: 24 samples
Clinical data shape: (1, 24)
Clinical data saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE201525.csv
Shape of linked data: (24, 31)
No valid trait values available, skipping missing value handling
Shape of linked data after handling missing values: (24, 31)
Dataset validation failed due to missing trait information. Final linked data not saved.


/media/techt/DATA/GenoAgent/tools/preprocess.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  linked_data = pd.concat([clinical_df, genetic_df], axis=0).T
